<a href="https://colab.research.google.com/github/cwcheng0/2019_Pixnet_hackathon/blob/master/2019Pixnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Read from Google
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error as MAE
import gc
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML


In [4]:
# Mount personal Google Drive; all data files are in personal Google Drive
# account: My main account @ Gmail
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
# Use PDrive function
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [6]:
# List files in root directory
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: PixnetData, id: 1zpLvHPbjbntAGzNfhYeQKo4I5PtIXmVw
title: Pixnet整理_Mike_20190804, id: 163PlI9NvQ5wVl62z6Crp4jihVWA4nnABRnPS8tU9neY
title: Tourism, id: 1rl2L8Vxmj4mqqFNCa23NUaypbVKazJRD
title: Python, id: 1dFGhOQOJ9t5x5bJUph24v_N-mnmBKsMZ
title: LionData, id: 192h03HmhseVUECi1YL9BAH44nzOAOl99
title: Colab Notebooks, id: 13yubYLLF_3_MObjXoj-K2aGo-_UFoQZo
title: bnbdata, id: 1yWUzpFHcZDftSVqbCaepJVd9oGsm2pww


In [7]:
# read data from an existing file to see if a command is correct?
f = open("/content/gdrive/My Drive/LionData/foo.txt", "r")
print(f.read())


Hello Google Drive!


In [8]:
# Read data
pd = pd.read_csv('/content/gdrive/My Drive/PixnetData/OriginalData/HotelsCombined API/Hackathon_Hotels_TW_EN.csv', header=None)

pd.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,ChainID,ChainName,CountryCode,CountryFileName,CountryName,CurrencyCode,Facilities,HotelAddress,HotelFileName,HotelID,HotelName,HotelPostcode,ImageID,Images,Latitude,Longitude,MinRate,NumberOfReviews,OverallRating,PlaceFileName,PlaceID,PlaceName,PlaceType,Popularity,PropertyType,PropertyTypeID,StarRating,StateName,StatePlacefilename,StatePlaceID,Themes,Trademarked
1,Null,NaN,TW,Taiwan,Taiwan,TWD,1|2|5|10|19|20|21|60|181|202|203|209|210|211,16 Kun Yuan Road,Friends_Yu_Pin_Hotel_Banciao_City,1652140,Friends Yu Pin Hotel Banciao City,NaN,Null,NaN,25.011741700000,121.460618600000,Null,5,6,New_Taipei,296823,New Taipei City,City,1875612,Hotel,0,2,NaN,NaN,Null,NaN,False
2,Null,NaN,TW,Taiwan,Taiwan,TWD,21|40|41|51|53|56|209|210|211,"3F, No.15, Ishin 2nd Road",Yes_Hotel_Kaohsiung,1652552,Yes Hotel Kaohsiung,NaN,Null,NaN,22.610372500000,120.312233000000,Null,0,Null,Kaohsiung,296791,Kaohsiung City,City,2163648,Hotel,0,2,NaN,NaN,Null,6,False
3,Null,NaN,TW,Taiwan,Taiwan,TWD,21|31|53|56|60,"No. 216, Dong-Wen Aw, Wenshan Rd.",Dolphin_Bay_Hotel_Makung,1652589,Dolphin Bay Hotel Makung,NaN,Null,NaN,23.559932500000,119.583457700000,Null,0,Null,Magong_City,48813,Magong City,Town,2315015,Hotel,0,4,NaN,NaN,Null,NaN,False
4,Null,NaN,TW,Taiwan,Taiwan,TWD,5|17|19|21|39|51|53|60|66|139|202|209|210|211,"No. 10, Ln. 291, Sec. 2, Daren Road, Wuci Dist...",A_Boutique_Hotel,1652613,A+ Boutique Hotel,NaN,Null,NaN,24.262567500000,120.536354100000,Null,0,Null,Taichung,296793,Taichung City,City,1901192,Hotel,0,3,NaN,NaN,Null,NaN,False


In [10]:
# Check first 5 rows in the targeted file.
import gzip
import pandas as pd

df2 = pd.read_json('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000005.gz', compression='gzip', lines=True)
  
#df2 = pd.read_csv('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000005.gz', compression='gzip', header=0, sep=' ', quotechar='"', error_bad_lines=False)                  
#df2 = pd.read_csv('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000005.gz', compression='gzip', header=0, sep=',', quotechar='"')

df2.head()

,author_id,browser,browser_version,category_id,city,cookie_age,cookie_gender,cookie_hash,country,date,device_brand,device_marketing,device_model,hour,os,os_version,referrer_host,referrer_url,referrer_venue,resolution,timestamp,url
0,wiki0918,Chrome,73.0.3683,財經企管,,不明,不明,rkLnxhYULJKA1N1rHRy0hQ==,TW,20190403,,,Other,23,Windows,,wiki0918.pixnet.net,http://wiki0918.pixnet.net/blog/post/164541912...,pixnet,1120x700,2019-04-03 15:18:22,http://wiki0918.pixnet.net/blog/post/164541912
1,ofeyhong,Chrome,75.0.3770,數位生活,Bade,不明,男,LbJV3CsF/57SsDbjuz7bhg==,TW,20190701,,,Other,23,Windows,,ofeyhong.pixnet.net,https://ofeyhong.pixnet.net/blog,pixnet,2560x1080,2019-07-01 15:05:08,https://ofeyhong.pixnet.net/blog/post/223618134
2,ryanyang0725,Chrome,73.0.3683,國內旅遊,,不明,不明,j+ftuAcka7hKI0DSwG9y4A==,TW,20190403,,,Other,14,Windows 7,,ryanyang0725.pixnet.net,http://ryanyang0725.pixnet.net/blog/post/21216...,pixnet,1093x615,2019-04-03 06:42:01,http://ryanyang0725.pixnet.net/blog/post/21216...
3,JessicaYiNingWu,Mobile Safari,12.0,國外旅遊,,不明,女,9hw6RDjGYiW+WxcfeCY36A==,TW,20190303,Apple,iPad,iPad,22,iOS,12.1.4,jessicayiningwu.pixnet.net,http://jessicayiningwu.pixnet.net/blog/post/22...,pixnet,834x1112,2019-03-03 14:28:07,http://jessicayiningwu.pixnet.net/blog/post/22...
4,queen010x,Chrome,73.0.3683,不設分類,Taipei,35-45,男,6u6MJHyoFoz5RTIThJaP7w==,TW,20190331,,,Other,11,Windows,,queen010x.pixnet.net,http://queen010x.pixnet.net/blog/post/27924126...,pixnet,1344x840,2019-03-31 03:18:19,http://queen010x.pixnet.net/blog/post/279241265


In [0]:
file_content = f.read()
  
with gzip.open('file.txt.gz', 'rb') as f:
  file_content = f.read()
  
df = gzip.open('/content/gdrive/My Drive/PixnetData/OriginalData/使用者瀏覽log/PIXNET_visit_log_000000000005.gz', 'wb')

df = pd.read_json("Hotelscombined_log_update", lines=True) 

pandas 
read_json(<檔案名>, lines=True) 

output = gzip.open('example_lines.txt.gz', 'wb')


file_content